Try to add a relationship to temperature extremes

In [1]:
from pysdmx.io import get_datasets
import utils
import json
import plotly.express as px


with open('sub_codelists.json', 'r') as f:
    sub_codelists = json.load(f)

def calculate_energy_consumption_per_capita(df):
    df['energy_consumption_PC_kwh'] = df['energy_consumption_gwh'] / df['population'] * 1000000
    return df

In [ ]:
average_household_constraints = {
    'geo': sub_codelists['geo_eu_countries']
}

demo_pjan_constraints = {
    'freq': 'A',
    'unit': 'NR',
    'age': 'TOTAL',
    'sex': 'T',
    'geo': sub_codelists['geo_eu_countries'],
    }